In [3]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_mass_shootings_in_the_United_States_in_2025'

In [12]:
email = "wella2@byu.edu"
ua = f"STAT386-class-assignment/1.0 (+{email})"

r = requests.get(url, headers={"User-Agent": ua, "From": email}, timeout=15)
r.raise_for_status()

tables = pd.read_html(r.text)

# Sanity check: how many tables did we pull?
print(f"Total tables found: {len(tables)}")

# Select the second table (index 1)
try:
    main_tbl = tables[1]
except IndexError:
    raise RuntimeError("There are fewer than 2 tables on this page — cannot extract the second one.")

print(main_tbl.head())
print(f"Rows: {len(main_tbl)}")
print("Columns:", main_tbl.columns.tolist())

Total tables found: 5
    2025 date         Location    State or territory Dead Injured  Total  \
0  November 2  Los Angeles (7)            California    0       4      4   
1  November 2       Cincinnati                  Ohio    0       4      4   
2  November 2   Washington (4)  District of Columbia    0       5      5   
3  November 2        Elk Grove            California    1       3      4   
4  November 2     Chicago (21)              Illinois    0       4      4   

                                         Description  
0  Four men were shot in the Winnetka neighborhoo...  
1  A dispute outside a nightclub in the Over-The-...  
2  Five people were shot in the Gateway neighborh...  
3  A shooting inside a bar killed a man and injur...  
4  Four people were shot in the Lake View neighbo...  
Rows: 383
Columns: ['2025 date', 'Location', 'State or territory', 'Dead', 'Injured', 'Total', 'Description']


/var/folders/f4/ftsj1j7n5837rlz4_q0rks500000gn/T/ipykernel_73557/4292007148.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
